
# Import the packages 

In [20]:
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
pd.set_option('display.max_colwidth', -1)

# Import data and extract consumer complaints column and product category column

In [2]:
data_new = pd.read_csv('Consumer_Complaints_new.csv')

C:\Users\USER\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (7,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data_new.columns

Index(['Date difference', 'Company public response',
       'Company response to consumer', 'Company', 'Complaint ID',
       'Consumer complaint narrative', 'Consumer consent provided?',
       'Consumer disputed?', 'Date received', 'Date sent to company', 'Issue',
       'Number of Records', 'Product_new', 'Product', 'State', 'Sub-issue',
       'Sub-product', 'Submitted via', 'Tags', 'Timely response?', 'ZIP code'],
      dtype='object')

In [4]:
df_pre = data_new.loc[data_new['Consumer complaint narrative'].notnull(), ['Consumer complaint narrative', 'Product_new']]

In [5]:
data_new['Product_new'] = pd.Categorical(data_new['Product_new'])
print (data_new['Product_new'].cat.categories)

Index(['Bank account or service', 'Checking or savings account',
       'Consumer Loan', 'Credit card or prepaid card',
       'Credit reporting or credit repair services', 'Debt collection',
       'Money transfer, virtual currency, or money service', 'Mortgage',
       'Other financial service', 'Payday loan, title loan, or personal loan',
       'Prepaid card', 'Student loan', 'Vehicle loan or lease'],
      dtype='object')


In [6]:
df_pre = df_pre.loc[df_pre['Product_new'].isin(['Credit reporting or credit repair services','Mortgage', 'Debt collection', 
                                                'Consumer Loan', 'Other financial service', 'Bank account or service'])]

In [7]:
df = df_pre.loc[df_pre['Product_new'].isin(['Bank account or service'])]

# Process text of consumer complaints

In [8]:
def pre_process(text):
    
    # lowercase
    text=text.lower()
    
    #remove tags
    text=re.sub("</?.*?>"," <> ",text)
    
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    return text

df['Consumer complaint narrative'] = df['Consumer complaint narrative'].apply(lambda x:pre_process(x))

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [9]:
def get_stop_words(stop_file_path):
    """load stop words """
    
    with open(stop_file_path, 'r', encoding="utf-8") as f:
        stopwords = f.readlines()
        stop_set = set(m.strip() for m in stopwords)
        return frozenset(stop_set)

#load a set of stop words
stopwords=get_stop_words("stopwords_v2.txt")

#get the text column 
docs=df['Consumer complaint narrative'].tolist()

#create a vocabulary of words, 
#ignore words that appear in 85% of documents, 
#eliminate stop words
cv=CountVectorizer(max_df=0.85,stop_words=stopwords)
word_count_vector=cv.fit_transform(docs)

# Extract keywords from text

In [10]:
cv=CountVectorizer(max_df=0.85,stop_words=stopwords,max_features=10000)
word_count_vector=cv.fit_transform(docs)
word_count_vector.shape

(14885, 10000)

In [11]:
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [12]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [13]:
feature_names=cv.get_feature_names()

# get the document that we want to extract keywords from
doc = ''.join(docs)

#generate tf-idf for the given document
tf_idf_vector=tfidf_transformer.transform(cv.transform([doc]))

#sort the tf-idf vectors by descending order of scores
sorted_items=sort_coo(tf_idf_vector.tocoo())

#extract only the top n; n here is 20
keywords=extract_topn_from_vector(feature_names,sorted_items,20)

# now print the results
print("\n===Keywords===")
for k in keywords:
    print(k,keywords[k])


===Keywords===
account 0.359
bank 0.289
check 0.171
money 0.153
card 0.144
told 0.143
credit 0.132
chase 0.121
fees 0.117
funds 0.11
checking 0.106
called 0.105
fargo 0.101
am 0.1
deposit 0.099
fee 0.097
time 0.097
overdraft 0.097
branch 0.094
accounts 0.093


# Find topics from text

In [14]:
vectorizer = TfidfVectorizer(stop_words='english', 
max_features= 1000, # keep top 1000 terms 
max_df = 0.5, 
smooth_idf=True)

X = vectorizer.fit_transform(df['Consumer complaint narrative'])

In [15]:
# SVD represent documents and terms in vectors 
svd_model = TruncatedSVD(n_components=20, algorithm='randomized', n_iter=100, random_state=122)

svd_model.fit(X)

TruncatedSVD(algorithm='randomized', n_components=20, n_iter=100,
       random_state=122, tol=0.0)

In [16]:
terms = vectorizer.get_feature_names()

for i, comp in enumerate(svd_model.components_):
    terms_comp = zip(terms, comp)
    sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=True)[:7]
    print("Topic "+str(i)+": ")
    for t in sorted_terms:
        print(t[0])
    print("===============")

Topic 0: 
xx
check
money
card
told
fees
credit
Topic 1: 
xx
bonus
promotion
citibank
citi
miles
requirements
Topic 2: 
bonus
promotion
citi
citibank
requirements
miles
offer
Topic 3: 
wells
fargo
bonus
promotion
opened
citi
requirements
Topic 4: 
overdraft
fees
fee
charged
wells
fargo
charge
Topic 5: 
check
deposit
deposited
funds
bonus
checks
cash
Topic 6: 
chase
wells
fargo
fees
overdraft
bonus
morgan
Topic 7: 
card
check
credit
debit
payment
report
overdraft
Topic 8: 
credit
accounts
payment
paypal
america
loan
closed
Topic 9: 
paypal
money
payment
funds
ebay
days
pay
Topic 10: 
america
paypal
funds
transactions
claim
fraud
chase
Topic 11: 
scottrade
information
overdraft
transactions
fees
charges
number
Topic 12: 
accounts
checking
deposit
savings
credit
funds
usaa
Topic 13: 
money
fees
scottrade
citibank
overdraft
usaa
miles
Topic 14: 
funds
payment
scottrade
deposit
usaa
available
transactions
Topic 15: 
fee
card
accounts
scottrade
funds
debit
usaa
Topic 16: 
scottrade
fee
money


# Extract the complaints related to products and company

In [17]:
df_pre = df_pre.loc[df_pre['Product_new'].isin(['Credit reporting or credit repair services','Mortgage', 'Debt collection', 
                                                'Consumer Loan', 'Other financial service', 'Bank account or service'])]

In [18]:
df = df_pre.loc[df_pre['Product_new'].isin(['Bank account or service'])]

In [19]:
df.loc[df['Consumer complaint narrative'].str.contains((" Citibank "))] 

,Consumer complaint narrative,Product_new
16827,"I have been a loyal customer of Citibank, and a Citigold member for years. On XXXX XXXX, funds in several of my accounts had been frozen, due to a name mismatch of the payee. The initial instructions given by the XXXX representative who contacted me were not accurate, as they asked me to bring a co-signor to the branch. The next morning, upon our arrival, I was told that my business partner could not co-sign because my account is frozen. If this was the case, why was this even presented as a solution to rectify the situation? My partner spent a workday in the bank with me and was n't even needed or able to assist in a viable solution. On XXXX/XXXX/16, after much time and effort, due to having to acquire a notarized affidavit, he completed and faxed an affidavit and was told that it would be resolved between 24-48 hours. As of XXXX XXXX, my account is still frozen. XXXX does not accept calls, and employees I contacted were increasingly rude and unhelpful, I resorted to tweeting XXXX, who discovered that the details from the fax were too dark to read. I do not understand why this was not communicated in a timely manner and that I had to inquire after waiting an extended period of time. Clearly, faxing is not the best way to send this information- they should have provided other options to send the affidavit and ensure a timely resolution. I now know that payments that came into my account had more than one name attached to them, and that to access my funds I will need each of these other persons to sign and send an affidavit. This is not a viable solution as the nature of my business makes this a nearly impossible feat. Because the bank allowed this to continue without issue for many months I was unaware there was an issue. I work with individuals across the United States and with many, I do not have a personal relationship simply a business relationship or contact. Obtaining an affidavit from each individual would be an enormous undertaking as these affidavits would have to be completed during regular business hours, due to the notarization requirement, and no compensation for the loss of time is available. \r\n\r\nI hold the opinion that I should have been privy to this information from the start. Since I previously had no issue with receiving payment in this manner for months, I had absolutely no idea what was happening, nor did I suspect that the outcome would leave me unable to access my funds, not only my earnings but those used to pay necessary associated business and living expenses. This is a systematic error and to freeze so many months of repeated payments at the click of a button, is unfair to the client especially when there is no viable solution and the institution is inflexible. This is a contrived issue as no court order was issued, no judgment rendered, nor fraud reported by any of the other named payees. I should have been told of any issues with my account from day one! As of the date of this complaint, the account is still frozen. I am both incredibly frustrated with the system, as well as with the staff of Citibank. On a number of occasions, the staff of Citibank was rude and provided misinformation in regard to courses of action. They repeatedly encouraged me to admit culpability for a situation that they created due to speculation instead of facts. Their conduct has made this terrible experience even worse. In many instances I had to call them, go into the bank, and go on twitter just to find out that there were problems that they had n't told me. I do not know when or if my account will be unfrozen, and can not access any of the money I 've worked so incredibly hard for. Their behavior is not only unethical but unconscionable and I respectfully request an investigation into their conduct and that you advocate for the rights of consumers like myself who have their money held hostage without sound reason",Bank account or service
18228,"I am writing a serious 